## Introduction

The idea is to have a twitter bot that allows users to tweet prompts @modernseinfeld_bot and receive a reply with the completion

The Twitter API requires that all requests use OAuth to authenticate. Once you sign up for twitter's developer platform https://developer.twitter.com/ you will get the required authentication credentials to be able to use the API. These credentials are three text strings:

- API key (token)
- API secret
- Bearer token

I store these in the .env file of the main project directory. This file is never checked into GitHub (by means of the .ignore file). We use load_dotenv() to load the required keys into our notebook.

In [89]:
import os
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv(override=True)
BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
CONSUMER_KEY = os.getenv('TWITTER_API_KEY')
CONSUMER_SECRET = os.getenv('TWITTER_API_SECRET')
ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
FINE_TUNED_MODEL = "curie:ft-personal-2022-02-16-02-07-10"

In [10]:
!curl -X GET -H "Authorization: Bearer {BEARER_TOKEN}" "https://api.twitter.com/2/tweets/20"

{"data":{"id":"20","text":"just setting up my twttr"}}

In [ ]:
!curl -X GET -H "Authorization: Bearer {BEARER_TOKEN}" "https://api.twitter.com/2/tweets/20?expansions=author_id"

In [41]:
!pip -qq install tweepy

In [66]:
import tweepy

# In latest version of tweepy (which I can't seem to install) #print(tweepy.__version__)
# there is a StreamingClient where you can add a rule
# e.g. streaming_client.add_rules(tweepy.StreamRule("@SeinfeldPlotBot"))
# We will use polling and keep a checkpoint file
# Also very important we will only be using Twitter API v2

# To connect as the user
client_user = tweepy.Client(
    consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET
)
# To connect as the app
client_app = tweepy.Client(BEARER_TOKEN)

In [94]:
!pip -qq install openai
import openai
def call_GPT3(prompt: str) -> str:
    """\
    Prompt our fine-tuned GPT-3 model and return the response.
    
    Args:
        prompt (str): the prompt part of the answer
    
    Returns:
        str: the prompt plus the completion from GPT3     
    """
    openai.api_key = OPENAI_KEY
    # We use same stop token from training
    STOP_TOKEN = " ##END##"

    # The fine tuned model is created by the fine tuning process above
    completion = openai.Completion.create(
        model=FINE_TUNED_MODEL,
        prompt=prompt,
        best_of = 8, #returns the "best" of 5 server-side completions (the one with the lowest log probability per token)
        n=1,
        max_tokens = 55)['choices'][0]
    
    return prompt + completion['text'].replace('\t','',-1).replace(str(STOP_TOKEN),"",-1)

In [100]:
def tweet_reply(text: str, tweet_id_to_reply: str) -> str:
    """\
    Reply to the mentioner with the text response.
    
    Args:
        text (str): the text of the tweet
        tweet_id_to_reply (str): the tweet to reply to
    
    Returns:
        str: the id of the reply tweet      
    """
    
    response = client_user.create_tweet(
        in_reply_to_tweet_id = tweet_id_to_reply,
        text=text
    )
    return f"https://twitter.com/user/status/{response.data['id']}"
    

In [85]:
# Get mentions of SeinfeldPlotBot
user_id = client_app.get_user(username='SeinfeldPlotBot').data.id
print("The SeinfeldPlotBot user_id is {}".format(user_id))
response = client.get_users_mentions(user_id, tweet_fields=["author_id"])

The SeinfeldPlotBot user_id is 1494776018313392130


In [103]:
for tweet in response.data:
    print(tweet.data)
    prompt = tweet.data['text'].lstrip("@SeinfeldPlotBot")
    completion = call_GPT3(prompt)
    response = tweet_reply(completion, tweet.data['id'])
    print(f"Replied with {completion} in this tweet: {response}")

{'author_id': '1494776018313392130', 'id': '1496299981627707395', 'text': '@SeinfeldPlotBot test'}
Replied with  test auditions for America's Next Top Model. Kramer tries to "stunt," but his friend the limo driver leaks his identity. ## in this tweet: https://twitter.com/user/status/1496592947797794826
